In [ ]:
import sys
import os

if os.getenv("CONTAINER"):
    path = '.'
else:
    path = '../../'

sys.path.insert(0, path)

In [ ]:
import tensorflow as tf
from models.snn.evaluate import evaluate_model
from models.preprocessing import load_preprocessed

In [ ]:
if os.getenv("CONTAINER"):
    print(tf.config.list_physical_devices("GPU"))

In [ ]:
search_hidden = [100, 200, 300]
search_syn = [0.005, 0.01, 0.02]

In [ ]:
best_score = -1
best_params = None

In [ ]:
splits, class_names = load_preprocessed(path + 'data/processed/')
X_val, y_val = splits['val']
X_test, y_test = splits['test']

In [ ]:
for h in search_hidden:
    for s in search_syn:
        acc, sim, history = evaluate_model(h, s, X_train=X_val, y_train=y_val, X_val=X_test, y_val=y_test, n_features=X_val.shape[1], n_classes=len(class_names))
        print(f"Tested hidden={h}, syn={s}, val_acc={acc:.3f}")

        if acc > best_score:
            best_score = acc
            best_params = (h, s)

        params_path = path + f'checkpoints/snn/hidden_{h}_syn_{s:.3f}.npz'

        sim.save_params(params_path)

        sim.close()

In [ ]:
print("Best:", best_params, "Acc:", best_score)

In [ ]:
with open(path + 'checkpoints/snn/.env', 'w') as f:
    f.write(f"HIDDEN_UNITS={best_params[0]}\n")
    f.write(f"SYNAPTIC_WEIGHT={best_params[1]}\n")